In [1]:
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet121
from keras.applications.resnet import ResNet50
from keras.applications.xception import Xception
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model

In [2]:
folder_name = "amanneo"
directory = f"..\\Datasets\\{folder_name}\\Splitted_Dataset_2"

trainDatagen = ImageDataGenerator(rescale= 1/255., rotation_range= 50, horizontal_flip= True, vertical_flip= True, validation_split=0.2)
trainData = trainDatagen.flow_from_directory(directory= f"{directory}\\train", batch_size=64, class_mode= 'categorical', target_size= (256, 256))

testDatagen = ImageDataGenerator(rescale= 1/255.)
testData = testDatagen.flow_from_directory(directory= f"{directory}\\test", batch_size=64, class_mode= 'categorical', target_size= (256, 256))

valData = trainDatagen.flow_from_directory(directory= f"{directory}\\val", batch_size=64, class_mode= 'categorical', target_size= (256, 256))

Found 21073 images belonging to 5 classes.
Found 7030 images belonging to 5 classes.
Found 7023 images belonging to 5 classes.


# DenseNet Model

In [3]:
# DenseNet

DenseNetBase = DenseNet121(weights='imagenet', include_top=False, input_shape=(256,256,3), classifier_activation='softmax')

layer = DenseNetBase.output
layer = GlobalAveragePooling2D()(layer)

layer = Dense(1024, activation='relu', kernel_initializer='he_normal')(layer)

DenseNetPreds = Dense(5, activation='softmax')(layer)

DenseNetModel = Model(inputs=DenseNetBase.input, outputs=DenseNetPreds)

print(f"DenseNet katman sayısı: {len(DenseNetModel.layers)}")


DenseNet katman sayısı: 430


In [4]:
for layer in DenseNetBase.layers:
    layer.trainable = False

for layer in DenseNetBase.layers[-100:]:
    layer.trainable = True

DenseNetModel.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
DenseNetModel.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 262, 262, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 128, 128, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                              

In [5]:
DenseNetHistory = DenseNetModel.fit(x=trainData, validation_data=valData, epochs=10)

Epoch 1/10
330/330 [==============================] - 239s 710ms/step - loss: 1.0887 - accuracy: 0.6516 - val_loss: 0.9177 - val_accuracy: 0.7332
Epoch 2/10
330/330 [==============================] - 230s 697ms/step - loss: 0.8948 - accuracy: 0.7327 - val_loss: 0.8830 - val_accuracy: 0.7333
Epoch 3/10
330/330 [==============================] - 227s 688ms/step - loss: 0.8740 - accuracy: 0.7339 - val_loss: 0.8716 - val_accuracy: 0.7323
Epoch 4/10
330/330 [==============================] - 227s 689ms/step - loss: 0.8627 - accuracy: 0.7339 - val_loss: 0.8637 - val_accuracy: 0.7333
Epoch 5/10
330/330 [==============================] - 228s 691ms/step - loss: 0.8491 - accuracy: 0.7342 - val_loss: 0.8535 - val_accuracy: 0.7349
Epoch 6/10
330/330 [==============================] - 228s 691ms/step - loss: 0.8394 - accuracy: 0.7342 - val_loss: 0.8469 - val_accuracy: 0.7343
Epoch 7/10
330/330 [==============================] - 231s 701ms/step - loss: 0.8363 - accuracy: 0.7345 - val_loss: 0.8464 -

KeyboardInterrupt: 

In [6]:
DenseNetModel.evaluate(testData)

110/110 [==============================] - 18s 158ms/step - loss: 0.8461 - accuracy: 0.7329


[0.8461034297943115, 0.732859194278717]

# ResNet

In [ ]:
ResNetBase = ResNet50(weights='imagenet', include_top=False, input_shape=(256,256,3), classifier_activation='softmax')

layer2 = ResNetBase.output

layer2 = GlobalAveragePooling2D()(layer2)

layer2 = Dense(1024, activation='relu')(layer2)

ResNetPreds = Dense(5, activation='softmax')(layer2)

ResNetModel = Model(inputs=ResNetBase.input, outputs=ResNetPreds)

print(f"ResNet katman sayısı: {len(ResNetModel.layers)}")

In [ ]:
for layer in ResNetBase.layers:
    layer.trainable = False

for layer in ResNetBase.layers[-10:]:
    layer.trainable = True

ResNetModel.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])

ResNetModel.summary()

In [ ]:
ResNetHistory = ResNetModel.fit(x=trainData, validation_data=valData, epochs=20)

In [ ]:
ResNetModel.evaluate(testData)

# Xception

In [ ]:
XceptionBase = Xception(weights='imagenet', include_top=False, input_shape=(256,256,3), classifier_activation='softmax')

x3 = XceptionBase.output
x3 = GlobalAveragePooling2D()(x3)

x3 = Dense(128, activation='relu')(x3)

XceptionPreds = Dense(3, activation='softmax')(x3)

XceptionModel = Model(inputs=XceptionBase.input, outputs=XceptionPreds)

print(f"Xception katman sayısı: {len(XceptionModel.layers)}")

In [ ]:
for layer in XceptionBase.layers:
    layer.trainable = False

for layer in XceptionBase.layers[-35:]:
    layer.trainable = True

XceptionModel.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
XceptionModel.summary()

In [ ]:
XceptionHistory = XceptionModel.fit(x = trainData, validation_data = valData, epochs = 50)

In [ ]:
XceptionModel.evaluate(testData)

In [ ]:
# 2x2 subplot grid'i oluşturun
fig, axs = plt.subplots(3, 2, figsize=(10, 10))

plt.suptitle("DenseNet - 100 Katman, ResNet - 10 Katman, Xception - 35 Katman Açık, AdaDelta Optimizer, Özel Epoch")

axs[0, 0].plot(DenseNetHistory.history['accuracy'], label='DenseNet Training Accuracy')
axs[0, 0].set_title('Training Accuracy')
axs[0, 0].legend()
axs[0, 0].plot(DenseNetHistory.history['val_accuracy'], label='DenseNet Validation Accuracy')
axs[0, 0].set_title('Validation Accuracy')
axs[0, 0].legend()

axs[0, 1].plot(DenseNetHistory.history['loss'], label='DenseNet Training Loss')
axs[0, 1].set_title('Training Loss')
axs[0, 1].legend()
axs[0, 1].plot(DenseNetHistory.history['val_loss'], label='DenseNet Validation Loss')
axs[0, 1].set_title('Validation Loss')
axs[0, 1].legend()

axs[1, 0].plot(ResNetHistory.history['accuracy'], label='ResNet Training Accuracy')
axs[1, 0].set_title('Training Accuracy')
axs[1, 0].legend()
axs[1, 0].plot(ResNetHistory.history['val_accuracy'], label='ResNet Validation Accuracy')
axs[1, 0].set_title('Validation Accuracy')
axs[1, 0].legend()

axs[1, 1].plot(ResNetHistory.history['loss'], label='ResNet Training Loss')
axs[1, 1].set_title('Training Loss')
axs[1, 1].legend()
axs[1, 1].plot(ResNetHistory.history['val_loss'], label='ResNet Validation Loss')
axs[1, 1].set_title('Validation Loss')
axs[1, 1].legend()

axs[2, 0].plot(XceptionHistory.history['accuracy'], label='Xception Training Accuracy')
axs[2, 0].set_title('Training Accuracy')
axs[2, 0].legend()
axs[2, 0].plot(XceptionHistory.history['val_accuracy'], label='Xception Validation Accuracy')
axs[2, 0].set_title('Validation Accuracy')
axs[2, 0].legend()

axs[2, 1].plot(XceptionHistory.history['loss'], label='Xception Training Loss')
axs[2, 1].set_title('Training Loss')
axs[2, 1].legend()
axs[2, 1].plot(XceptionHistory.history['val_loss'], label='Xception Validation Loss')
axs[2, 1].set_title('Validation Loss')
axs[2, 1].legend()

# Grafikleri düzenli hale getirin
plt.tight_layout()
plt.show()